In [2]:
#limpa df, comeca etc

import pandas as pd
import numpy as np
import re

df = pd.read_csv('data/IMDB.csv' , encoding='utf-8')

def clean_t(t):
    t_clean = re.sub(r'<.*?>', '', t)
    t_clean = re.sub(r'[^\w\s]', '', t_clean)
    return t_clean

df['review'] = df['review'].apply(clean_t)
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production The filming tech...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically theres a family where a little boy J...,negative
4,Petter Matteis Love in the Time of Money is a ...,positive


In [7]:
#fazendo o treino
df_treino = df.iloc[:500]
df_treino.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production The filming tech...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically theres a family where a little boy J...,negative
4,Petter Matteis Love in the Time of Money is a ...,positive


In [8]:
#fazendo a dict de palavras nao repetidas
# primeiro bullet point
df_positivo = df_treino[df_treino['sentiment'] == 'positive']
df_negativo = df_treino[df_treino['sentiment'] == 'negative']
numero_t_positivo = len(df_positivo)
numero_t_negativo = len(df_negativo)

# Inicializa os dicionários de contagem para positivos e negativos
contagem_palavras = {}

# Itera sobre os textos e sentimentos
for texto, sentimento in zip(df_treino['review'], df_treino['sentiment']):
    palavras = set(texto.lower().split())  # Divide o texto em palavras
    for palavra in palavras:
        if palavra not in contagem_palavras:
            contagem_palavras[palavra] = {'positive': 0, 'negative': 0}
        contagem_palavras[palavra][sentimento] += 1


prob_palavra_dado_classe = {}

for palavra2,dictt in contagem_palavras.items():
    pos = (dictt['positive']) / (numero_t_positivo)
    nega = (dictt['negative']) / (numero_t_negativo)
    prob_palavra_dado_classe[palavra2] = {'positive': pos, 'negative': nega}


prob_not_palavra_dado_classe = {}

for palavra3,dictt2 in prob_palavra_dado_classe.items():
    pos2 = 1 - dictt2['positive']
    nega2 = 1 - dictt2['negative']
    prob_not_palavra_dado_classe[palavra3] = {'positive': pos2, 'negative': nega2}


{'italians': {'positive': 1, 'negative': 0}, 'hardcore': {'positive': 3, 'negative': 0}, 'of': {'positive': 226, 'negative': 252}, 'its': {'positive': 114, 'negative': 129}, 'order': {'positive': 11, 'negative': 8}, 'on': {'positive': 158, 'negative': 168}, 'sex': {'positive': 6, 'negative': 16}, 'happened': {'positive': 7, 'negative': 6}, 'out': {'positive': 113, 'negative': 120}, 'pulls': {'positive': 3, 'negative': 1}, 'audiences': {'positive': 6, 'negative': 3}, 'from': {'positive': 125, 'negative': 118}, 'got': {'positive': 31, 'negative': 35}, 'exactly': {'positive': 10, 'negative': 9}, 'surreal': {'positive': 1, 'negative': 0}, 'goes': {'positive': 18, 'negative': 27}, 'was': {'positive': 143, 'negative': 175}, 'experimental': {'positive': 2, 'negative': 1}, 'regards': {'positive': 3, 'negative': 0}, 'nickname': {'positive': 1, 'negative': 1}, 'more': {'positive': 100, 'negative': 87}, 'fact': {'positive': 14, 'negative': 34}, 'inwards': {'positive': 1, 'negative': 0}, 'bitches'

In [ ]:
#estimando P(palavra)
#segundo bullet point
prob_palavra = {}


In [14]:
df_teste = df.iloc[25000:]
df_teste.head()

,review,sentiment
25000,This movie was bad from the start The only pur...,negative
25001,God I never felt so insulted in my whole life ...,negative
25002,Not being a fan of the Coen Brothers or George...,positive
25003,The movie Andaz Apna Apna in my books is the t...,positive
25004,I have to say I was really looking forward on ...,negative
